In [ ]:
%sql
drop database if exists country_club cascade;
create database country_club;
show databases;

databaseName
country_club
default


In [ ]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

In [ ]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
 |-- bookid: integer (nullable = true)
 |-- facid: integer (nullable = true)
 |-- memid: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- slots: integer (nullable = true)

Facilities Schema
root
 |-- facid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- membercost: double (nullable = true)
 |-- guestcost: double (nullable = true)
 |-- initialoutlay: integer (nullable = true)
 |-- monthlymaintenance: integer (nullable = true)

Members Schema
root
 |-- memid: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- address: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- telephone: string (nullable = true)
 |-- recommendedby: integer (nullable = true)
 |-- joindate: timestamp (nullable = true)



In [ ]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

In [ ]:
%sql
use country_club;
REFRESH table bookings;
REFRESH table facilities;
REFRESH table members;
show tables;

database,tableName,isTemporary
country_club,bookings,false
country_club,facilities,false
country_club,members,false


In [ ]:
%sql
select * from bookings limit 3

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03T11:00:00.000+0000,2
1,4,1,2012-07-03T08:00:00.000+0000,2
2,6,0,2012-07-03T18:00:00.000+0000,2


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [ ]:
%sql
SELECT * FROM FACILITIES 
WHERE membercost <> 0

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5.0,25.0,10000,200
1,Tennis Court 2,5.0,25.0,8000,200
4,Massage Room 1,9.9,80.0,4000,3000
5,Massage Room 2,9.9,80.0,4000,3000
6,Squash Court,3.5,17.5,5000,80


####  Q2: How many facilities do not charge a fee to members?

In [ ]:
%sql
SELECT count(*) FROM FACILITIES 
WHERE membercost = 0

count(1)
4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [ ]:
%sql
select 
  facid,
  name,
  membercost,
  monthlymaintenance,
  (monthlymaintenance*.20)
  
from facilities
where membercost <> 0
and (membercost < monthlymaintenance*.20)

facid,name,membercost,monthlymaintenance,(monthlymaintenance * 0.20)
0,Tennis Court 1,5.0,200,40.00
1,Tennis Court 2,5.0,200,40.00
4,Massage Room 1,9.9,3000,600.00
5,Massage Room 2,9.9,3000,600.00
6,Squash Court,3.5,80,16.00


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [ ]:
%sql
select * from facilities
where facid in (1,5)

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [ ]:
%sql
select
  NAME,
  (
    CASE
      WHEN monthlymaintenance > 100 THEN 'Expensive'
      ELSE 'Cheap'
    END
  ) AS Monthly_Maintenance,
  monthlymaintenance
from
  facilities

NAME,Monthly_Maintenance,monthlymaintenance
Tennis Court 1,Expensive,200
Tennis Court 2,Expensive,200
Badminton Court,Cheap,50
Table Tennis,Cheap,10
Massage Room 1,Expensive,3000
Massage Room 2,Expensive,3000
Squash Court,Cheap,80
Snooker Table,Cheap,15
Pool Table,Cheap,15


d
#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [ ]:
%sql
select firstname, surname from members
where joindate in (select max(joindate) from members)


firstname,surname
Darren,Smith


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [ ]:

%sql
select distinct f.name, concat(m.firstname," ", m.surname) as MemberName from bookings b
inner join facilities f on b.facid = f.facid
inner join members m on b.memid = m.memid
where f.name like 'Tennis Court%'
order by MemberName

name,MemberName
Tennis Court 2,Anne Baker
Tennis Court 1,Anne Baker
Tennis Court 2,Burton Tracy
Tennis Court 1,Burton Tracy
Tennis Court 1,Charles Owen
Tennis Court 2,Charles Owen
Tennis Court 2,Darren Smith
Tennis Court 2,David Farrell
Tennis Court 1,David Farrell
Tennis Court 2,David Jones


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [ ]:
%sql

select 
    f.name as FacilityName, 
    concat(firstname," ", surname) as Name,
    
    (
        case 
            when 
                m.memid=0 
            then f.guestcost*b.slots
    
            else 
                f.membercost*b.slots
        end
    ) as TotalCost

from 
    bookings b 
    join members m on b.memid=m.memid
    join facilities f on b.facid=f.facid

where 
    starttime >= '2012-09-14' and starttime <= '2012-09-15'
and 
    (
        (m.memid=0 and (f.guestcost*b.slots) >30 ) 
        or 
        (m.memid <>0 and (f.membercost*b.slots)> 30)
    )
order by TotalCost desc

FacilityName,Name,TotalCost
Massage Room 2,GUEST GUEST,320.0
Massage Room 1,GUEST GUEST,160.0
Massage Room 1,GUEST GUEST,160.0
Massage Room 1,GUEST GUEST,160.0
Tennis Court 2,GUEST GUEST,150.0
Tennis Court 2,GUEST GUEST,75.0
Tennis Court 1,GUEST GUEST,75.0
Tennis Court 1,GUEST GUEST,75.0
Squash Court,GUEST GUEST,70.0
Massage Room 1,Jemima Farrell,39.6


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [ ]:
%sql

Select 

sub.FacilityName, sub.Name, sub.TotalCost

from (
select 
    f.name as FacilityName, 
    starttime,
    concat(firstname," ", surname) as Name,
    
    (
        case 
            when 
                m.memid=0 
            then f.guestcost*b.slots
    
            else 
                f.membercost*b.slots
        end
    ) as TotalCost

from 
    bookings b 
    join members m on b.memid=m.memid
    join facilities f on b.facid=f.facid

where 

    (
        (m.memid=0 and (f.guestcost*b.slots) >30 ) 
        or 
        (m.memid <>0 and (f.membercost*b.slots)> 30)
    )
) sub

where     sub.starttime >= '2012-09-14' and sub.starttime <= '2012-09-15'
order by sub.TotalCost desc
 

FacilityName,Name,TotalCost
Massage Room 2,GUEST GUEST,320.0
Massage Room 1,GUEST GUEST,160.0
Massage Room 1,GUEST GUEST,160.0
Massage Room 1,GUEST GUEST,160.0
Tennis Court 2,GUEST GUEST,150.0
Tennis Court 2,GUEST GUEST,75.0
Tennis Court 1,GUEST GUEST,75.0
Tennis Court 1,GUEST GUEST,75.0
Squash Court,GUEST GUEST,70.0
Massage Room 1,Jemima Farrell,39.6


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [ ]:
%sql

select * from 

(
    select 
        f.name as FacilityName, 
        sum((
            case 
                when 
                    m.memid=0 
                then f.guestcost*b.slots
        
                else 
                    f.membercost*b.slots
            end
        )) as TotalRevenue

    from 
        bookings b 
        join members m on b.memid=m.memid
        join facilities f on b.facid=f.facid

    group by FacilityName

) sub

where sub.TotalRevenue < 1000

FacilityName,TotalRevenue
Pool Table,270.0
Table Tennis,180.0
Snooker Table,240.0
